# Running RRTMGP on Garand Atmospheres

## File Staging

In [7]:
import os, sys, shutil
import subprocess as sub

sys.path.append('common')
import utils

In [13]:
sw = False
iCost = 1
iNorm = 1
iteration = 1
gpt1 = 54

domain = 'sw' if sw else 'lw'
costStr = '{}.cost{:02d}.norm{:1d}'.format(domain, iCost, iNorm)
pScratch = '/global/cscratch1/sd/pernak18/RRTMGP/g-point-reduction/data'
project = '/project/projectdirs/e3sm/pernak18/g-point-reduction/k-distribution-opt'
coeffNC = '{}/{}/coefficients_iter{:03d}.{:03d}.{:03d}.nc'.format(
    pScratch, costStr, iteration, gpt1, gpt1+1)

exe = '{}/rrtmgp_garand_atmos'.format(project)

paths = [coeffNC, exe]

# link to k-distribution for this iteration/g-point combination
kDistNC = 'coefficients_{}.nc'.format(domain)
if os.path.islink(kDistNC): os.unlink(kDistNC)
os.symlink(coeffNC, kDistNC)

# copy LBLRTM reference netCDF with Garand specs
# 'test_RRTMGP_fluxes.nc' will be overwritten if it already exists
garandNC = '/project/projectdirs/e3sm/pernak18/inputs/g-point-reduce/' + \
    'lblrtm-lw-flux-inputs-outputs-garandANDpreind.nc'
outNC = 'test_RRTMGP_fluxes.nc'
shutil.copyfile(garandNC, outNC)

# link to Garand atmosphere driver for RRTMGP
baseExe = os.path.basename(exe)
if os.path.islink(baseExe): os.unlink(baseExe)
os.symlink(exe, baseExe)

status = sub.call(['./{}'.format(baseExe), garandNC, kDistNC])
if status == 0: print('RRTMGP fluxes have been written to {}'.format(
    os.path.abspath(outNC)))

RRTMGP fluxes have been written to test_RRTMGP_fluxes.nc
